In [7]:
import os
trees=["NOMINAL"]
with open("../MC/sys_trees.txt","r") as file:
    line=file.readline()
    while line:
        trees.append(line.rstrip('\n'))
        line=file.readline()

samples=["Signal_Sherpa","Signal_PoPy","Zmumu","VV"]

for i in samples:
    for t in trees:
        os.system("cp ../MC/out/"+i+t+".root .")

In [53]:
import ROOT as r
dict={}

# FILLING DICTIONARY WITH SYS-RELATED TREES 

### the dictionary will have the following structure "systematic_name":[fake1p,truth1p,fake3p,truth3p]

for sam in samples:
    for sys in trees[1:]:
        file=r.TFile.Open(sam+sys+".root")
        prong1=file.Get("tau_matched_1p_topo_dphi_btag_iso_rnn_ptmu_omega_mreco_tpt")
        prong3=file.Get("tau_matched_3p_topo_dphi_btag_iso_rnn_ptmu_omega_mreco_tpt")
        dict[sam+sys]=[prong1.GetBinContent(1),prong1.GetBinContent(2),prong3.GetBinContent(1),prong3.GetBinContent(2)]
        file.Close()

#NOW ADDING THE WEIGHT-BASED SYSTEMATICS        
histograms=[]
for sys in trees[:1]:
    file=r.TFile.Open("Signal_Sherpa"+sys+".root")
    histos=file.GetListOfKeys()
    for i in histos:
        name=i.GetName()
        if ("cuts" in name) and ("tpt" in name) and ("1p" in name):
            histograms.append(i.GetName()[len("tau_matched_1p_cuts_tpt_"):])

for sam in samples:
    for sys in trees[:1]:
        file=r.TFile.Open(sam+sys+".root")
        for h in histograms:
            prong1=file.Get("tau_matched_1p_cuts_tpt_"+h)
            prong3=file.Get("tau_matched_3p_cuts_tpt_"+h)
            dict[sam+sys+h]=[prong1.GetBinContent(1),prong1.GetBinContent(2),prong3.GetBinContent(1),prong3.GetBinContent(2)]
        file.Close()

In [54]:
# REARRANGING THE DICTIONARY AS FAKES AND TRUTH EVENTS
for i in dict:
    if "Signal" not in i:
        dict[i]=[dict[i][0]+dict[i][1],0,dict[i][2]+dict[i][3],0]

In [55]:
#The structure of the dictionary is... "name_of_sys":[SherpaFake,SherpaTruth,PoPyFake,PoPyTruth,Zmumu,VV]
final_dict_1p={}
for i in histograms+trees[1:]:
    final_dict_1p[i]=[0,0,0,0,0,0]
    for j in dict:
        if (i in j) and ("Sherpa" in j):
            final_dict_1p[i][0]=dict[j][0]
            final_dict_1p[i][1]=dict[j][1]
        if (i in j) and ("PoPy" in j):
            final_dict_1p[i][2]=dict[j][0]
            final_dict_1p[i][3]=dict[j][1]
        if (i in j) and ("Zmumu" in j):
            final_dict_1p[i][4]=dict[j][0]
        if (i in j) and ("VV" in j):
            final_dict_1p[i][5]=dict[j][0]
            
final_dict_3p={}
for i in histograms+trees[1:]:
    final_dict_3p[i]=[0,0,0,0,0,0]
    for j in dict:
        if (i in j) and ("Sherpa" in j):
            final_dict_3p[i][0]=dict[j][2]
            final_dict_3p[i][1]=dict[j][3]
        if (i in j) and ("PoPy" in j):
            final_dict_3p[i][2]=dict[j][2]
            final_dict_3p[i][3]=dict[j][3]
        if (i in j) and ("Zmumu" in j):
            final_dict_3p[i][4]=dict[j][2]
        if (i in j) and ("VV" in j):
            final_dict_3p[i][5]=dict[j][2]

#TAKING DOWN AND UP SYSTEMATICS TO THE BOTTOM AND THE TOP OF THE DICTIONARY RESPECTIVELY
down_1p={}
down_3p={}

a1p_keys=[i for i in final_dict_1p]
a3p_keys=[i for i in final_dict_3p]

for i in a1p_keys:
    if "1down" in i:
        down_1p.update({i:final_dict_1p[i]})
        final_dict_1p.pop(i)
for i in a3p_keys:
    if "1down" in i:
        down_3p.update({i:final_dict_3p[i]})
        final_dict_3p.pop(i)
final_dict_1p.update(down_1p)
final_dict_3p.update(down_3p)

In [56]:
# EXPORTING THE RESULTS TO A CSV FILE
import pandas as pd
df1=pd.DataFrame.from_dict(final_dict_1p, orient='index',
                       columns=["SherpaFake","SherpaTruth","PoPyFake","PoPyTruth","Zmumu","VV"])
df3=pd.DataFrame.from_dict(final_dict_3p, orient='index',

                       columns=["SherpaFake","SherpaTruth","PoPyFake","PoPyTruth","Zmumu","VV"])

df1.to_csv('Weight-Based-Uncertainties-1prong.csv')
df3.to_csv('Weight-Based-Uncertainties-3prong.csv')


In [57]:
df1

,SherpaFake,SherpaTruth,PoPyFake,PoPyTruth,Zmumu,VV
muon_0_MUON_EFF_ISO_STAT_1up_MuEffSF_IsoLoose_FixedRad,122.927704,10020.984375,103.185585,11071.424805,145.444561,163.402431
muon_0_MUON_EFF_ISO_STAT_1up_MuEffSF_IsoTightTrackOnly_FixedRad,122.951431,10023.425781,103.198296,11074.015625,145.474289,163.448391
muon_0_MUON_EFF_ISO_SYS_1up_MuEffSF_IsoLoose_FixedRad,123.045143,10029.595703,103.273811,11080.930664,145.525722,163.530915
muon_0_MUON_EFF_ISO_SYS_1up_MuEffSF_IsoTightTrackOnly_FixedRad,123.661797,10070.153320,103.711555,11127.401367,145.938819,164.093807
muon_0_MUON_EFF_RECO_STAT_1up_MuEffSF_Reco_QualMedium,122.965485,10024.264648,103.228935,11075.145508,145.482647,163.451995
...,...,...,...,...,...,...
TAUS_TRUEHADTAU_SME_TES_DETECTOR_1down,123.530373,10164.585938,103.175766,11283.042969,145.945478,166.737689
TAUS_TRUEHADTAU_SME_TES_INSITUEXP_1down,122.965042,9936.470703,103.175766,10994.750000,145.945478,162.672901
TAUS_TRUEHADTAU_SME_TES_INSITUFIT_1down,122.965042,9848.962891,103.175766,10922.850586,145.945478,161.794254
TAUS_TRUEHADTAU_SME_TES_MODEL_CLOSURE_1down,122.965042,9993.267578,103.175766,11069.314453,145.945478,163.564258
